In [113]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

GRADE_MAP = {
    "completed-auctions-used": "Ungraded",
    "completed-auctions-grade-nineteen": "CGC 10 Prist.",
    "completed-auctions-manual-only": "PSA 10",
    "completed-auctions-grade-seventeen": "CGC 10",
    "completed-auctions-grade-twenty-one": "TAG 10",
    "completed-auctions-grade-twenty-two": "ACE 10",
    "completed-auctions-box-only": "Grade 9.5",
    "completed-auctions-graded": "Grade 9",
    "completed-auctions-new": "Grade 8",
    "completed-auctions-cib": "Grade 7",
    "completed-auctions-grade-six": "Grade 6",
    "completed-auctions-grade-five": "Grade 5",
    "completed-auctions-grade-four": "Grade 4",
    "completed-auctions-grade-three": "Grade 3",
    "completed-auctions-box-and-manual": "Grade 2",
    "completed-auctions-loose-and-manual": "Grade 1",
}

In [114]:
url = "https://www.pricecharting.com/game/pokemon-ruby-&-sapphire/blaziken-reverse-holo-3"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

In [118]:
all_sales = []

# IMPORTANT FIX:
# - When class_=callable is used, BeautifulSoup often passes the class attribute as a STRING.
# - So match on substring "completed-auctions-" at this stage.
containers = soup.select("div[class*='completed-auctions-']")

for container in containers:
    class_list = container.get("class") or []

    # Find the specific completed-auctions-* class from the list
    container_class = next((c for c in class_list if c.startswith("completed-auctions-")), None)
    if not container_class:
        continue

    grade_label = GRADE_MAP.get(container_class)
    if not grade_label:
        continue

    rows = container.select("tr[id^='ebay-']")

    if not rows:
        continue

    print(container_class, "->", len(rows))

    for row in rows:
        title_tag = row.find("td", class_="title")
        price_tag = row.find("span", class_="js-price")
        date_tag = row.find("td", class_="date")

        if not (title_tag and price_tag and date_tag):
            continue

        ebay_id = (row.get("id") or "").replace("ebay-", "")

        all_sales.append({
            "ebay_id": ebay_id,
            "grade": grade_label,
            "container": container_class,
            "date_raw": date_tag.get_text(strip=True),
            "title": title_tag.get_text(strip=True),
            "price_raw": price_tag.get_text(strip=True),
        })

df = pd.DataFrame(all_sales)

# Clean price + date
df["price"] = (
    df["price_raw"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
    .astype(float)
)
df["date"] = pd.to_datetime(df["date_raw"])

df = df[["ebay_id", "grade", "date", "price", "title", "container"]]

print("\nTotal sales:", len(df))
print(df["grade"].value_counts())

df.head()

completed-auctions-used -> 29
completed-auctions-cib -> 6
completed-auctions-new -> 7
completed-auctions-graded -> 24
completed-auctions-box-only -> 2
completed-auctions-manual-only -> 1
completed-auctions-loose-and-manual -> 1
completed-auctions-grade-three -> 1
completed-auctions-grade-five -> 3
completed-auctions-grade-six -> 3

Total sales: 77
grade
Ungraded     29
Grade 9      24
Grade 8       7
Grade 7       6
Grade 5       3
Grade 6       3
Grade 9.5     2
PSA 10        1
Grade 1       1
Grade 3       1
Name: count, dtype: int64


,ebay_id,grade,date,price,title,container
0,168169523626,Ungraded,2026-02-25,43.00,Pokemon TCG EX Ruby & Sapphire Blaziken Revers...,completed-auctions-used
1,366111006668,Ungraded,2026-01-26,28.47,Pokemon TCG English Card ex Ruby & Sapphire Bl...,completed-auctions-used
2,235472076109,Ungraded,2026-01-26,39.98,3/109 | Blaziken Reverse Holo | EX Ruby Sapphi...,completed-auctions-used
3,205558954522,Ungraded,2026-01-19,7.61,Blaziken Reverse Holo Rare 3/109 Ruby & Sapphi...,completed-auctions-used
4,354530715182,Ungraded,2026-01-19,27.99,Blaziken Reverse Holo Rare - 2003 Pokemon Ex R...,completed-auctions-used


In [122]:
from app.services.pricecharting import scrape_pricecharting_sales

url = "https://www.pricecharting.com/game/pokemon-ruby-&-sapphire/blaziken-reverse-holo-3"

df = scrape_pricecharting_sales(url)

print(len(df))
df.head()

77


,ebay_id,grade,date,price,title
0,168169523626,Ungraded,2026-02-25,43.00,Pokemon TCG EX Ruby & Sapphire Blaziken Revers...
1,366111006668,Ungraded,2026-01-26,28.47,Pokemon TCG English Card ex Ruby & Sapphire Bl...
2,235472076109,Ungraded,2026-01-26,39.98,3/109 | Blaziken Reverse Holo | EX Ruby Sapphi...
3,205558954522,Ungraded,2026-01-19,7.61,Blaziken Reverse Holo Rare 3/109 Ruby & Sapphi...
4,354530715182,Ungraded,2026-01-19,27.99,Blaziken Reverse Holo Rare - 2003 Pokemon Ex R...
